In [34]:
%load_ext autoreload
%autoreload 2

import numpy as np, h5py

%config Completer.use_jedi = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
from SelectionFunctionGaiaArchive import GaiaArchiveQuery

table="gaiaedr3"; mag_res=0.1; col_res=0.5; pos_level=7
query = GaiaArchiveQuery(magnitude_column = 'phot_g_mean_mag',
                         magnitude_resolution = mag_res,
                         colour_column = 'bp_rp',
                         colour_resolution = col_res,
                         position_column='source_id',
                         position_resolution=pos_level,
                         superset_criteria =['phot_rp_mean_mag>6','phot_rp_mean_mag < 16'],
                         subset_criteria=['dr2_radial_velocity > -1000'])

In [36]:
query.query

'select magnitude, colour, position, count(*) as n, sum(selection) as k \n from ( select  to_integer(floor(phot_g_mean_mag * 10)) as magnitude, \nto_integer(floor(bp_rp * 2)) as colour, \ngaia_healpix_index(7, source_id) as position, \nto_integer(IF_THEN_ELSE(dr2_radial_velocity > -1000, 1.0, 0.0)) as selection from gaiaedr3.gaia_source \n where phot_rp_mean_mag>6 and phot_rp_mean_mag < 16  ) as subquery \n group by magnitude, colour, position'

In [6]:
query.run_query()

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Running query
INFO: Query finished. [astroquery.utils.tap.core]
Finished query, it took 2338.9 seconds


In [32]:
with h5py.File(f'/data/asfe2/Projects/astrometry/{table}_rvs_kncounts.h', 'w') as hf:
    for key in ['magnitude', 'colour', 'position', 'k', 'n']:
        hf.create_dataset(key, data=np.array(query.results[key]))
    
    hf.create_dataset('magnitude_res', data=np.array([0.1]))
    hf.create_dataset('colour_res', data=np.array([0.5]))
    hf.create_dataset('position_level', data=np.array([7]))